In [1]:
def gaussian_elimination(A, b):
    """
    Solves the system of linear equations Ax = b using Gaussian elimination.

    Parameters:
    A (list of lists of floats): The coefficient matrix.
    b (list of floats): The right-hand side vector.

    Returns:
    list of floats: The solution vector x.
    """
    n = len(A)
    # Augment the matrix A with the vector b
    for i in range(n):
        A[i].append(b[i])

    # Keep track of row swaps
    swap_tracker = list(range(n))

    # Forward elimination
    for i in range(n):
        # Find the maximum element in the current column
        max_el = abs(A[i][i])
        max_row = i
        for k in range(i + 1, n):
            if abs(A[k][i]) > max_el:
                max_el = abs(A[k][i])
                max_row = k

        # Swap the maximum row with the current row
        A[i], A[max_row] = A[max_row], A[i]
        swap_tracker[i], swap_tracker[max_row] = swap_tracker[max_row], swap_tracker[i]

        # Make all rows below this one 0 in the current column
        for k in range(i + 1, n):
            c = -A[k][i] / A[i][i]
            for j in range(i, n + 1):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += c * A[i][j]

    # Backward substitution
    x = [0 for _ in range(n)]
    for i in range(n - 1, -1, -1):
        x[i] = A[i][n] / A[i][i]
        for k in range(i - 1, -1, -1):
            A[k][n] -= A[k][i] * x[i]

    # Restore the original order of the solution vector
    solution = [0 for _ in range(n)]
    for i in range(n):
        solution[swap_tracker[i]] = x[i]

    return solution

# Example usage:
A = [[2, 1, -1],
     [-3, -1, 2],
     [-2, 1, 2]]

b = [8, -11, -3]

solution = gaussian_elimination(A, b)
print("Solution:", solution)


Solution: [-0.9999999999999999, 2.0, 3.0000000000000004]


In [2]:
import numpy as np

def gaussian_elimination_np(A, b):
    """
    Solves the system of linear equations Ax = b using Gaussian elimination.

    Parameters:
    A (numpy.ndarray): The coefficient matrix.
    b (numpy.ndarray): The right-hand side vector.

    Returns:
    numpy.ndarray: The solution vector x.
    """
    n = len(A)
    # Augment the matrix A with the vector b
    Ab = np.hstack((A, b.reshape(-1, 1)))

    # Keep track of row swaps
    swap_tracker = np.arange(n)

    # Forward elimination
    for i in range(n):
        # Find the maximum element in the current column
        max_row = np.argmax(np.abs(Ab[i:, i])) + i
        
        # Swap the maximum row with the current row
        Ab[[i, max_row]] = Ab[[max_row, i]]
        swap_tracker[[i, max_row]] = swap_tracker[[max_row, i]]

        # Make all rows below this one 0 in the current column
        for k in range(i + 1, n):
            c = -Ab[k, i] / Ab[i, i]
            Ab[k, i:] += c * Ab[i, i:]

    # Backward substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (Ab[i, n] - np.dot(Ab[i, i+1:n], x[i+1:n])) / Ab[i, i]

    # Restore the original order of the solution vector
    solution = np.zeros(n)
    for i in range(n):
        solution[swap_tracker[i]] = x[i]

    return solution

# Example usage:
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]], dtype=float)

b = np.array([8, -11, -3], dtype=float)

solution = gaussian_elimination_np(A, b)
print("Solution:", solution)


Solution: [-1.  2.  3.]


In [3]:
import numpy as np

def calculate_determinant(A):
    """
    Calculates the determinant of a square matrix A using Gaussian elimination.

    Parameters:
    A (numpy.ndarray): The square matrix.

    Returns:
    float: The determinant of the matrix A.
    """
    n = A.shape[0]
    A = A.copy()  # Make a copy of A to avoid modifying the original matrix
    det = 1.0
    sign_change = 1

    # Forward elimination
    for i in range(n):
        # Find the maximum element in the current column
        max_row = np.argmax(np.abs(A[i:, i])) + i
        
        # If the maximum element is zero, the determinant is zero
        if A[max_row, i] == 0:
            return 0
        
        # Swap the maximum row with the current row
        if max_row != i:
            A[[i, max_row]] = A[[max_row, i]]
            sign_change *= -1  # Keep track of row swaps

        # Multiply the determinant by the pivot element
        det *= A[i, i]

        # Make all rows below this one 0 in the current column
        for k in range(i + 1, n):
            c = -A[k, i] / A[i, i]
            A[k, i:] += c * A[i, i:]

    # Apply the sign change due to row swaps
    det *= sign_change

    return det

# Example usage:
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]], dtype=float)

det = calculate_determinant(A)
print("Determinant:", det)


Determinant: -0.9999999999999993


In [4]:
import numpy as np

def lu_decomposition_with_pivoting(A):
    """
    Performs LU decomposition of a square matrix A with partial pivoting.
    The decomposition is such that P*A = L*U where
    P is a permutation matrix, L is a lower triangular matrix, and U is an upper triangular matrix.

    Parameters:
    A (numpy.ndarray): The square matrix to decompose.

    Returns:
    tuple: A tuple containing P, L, and U matrices.
    """
    n = A.shape[0]
    A = A.copy()  # Make a copy of A to avoid modifying the original matrix
    L = np.zeros((n, n), dtype=float)
    U = np.zeros((n, n), dtype=float)
    P = np.eye(n)  # Permutation matrix

    for i in range(n):
        # Partial pivoting
        max_row = np.argmax(np.abs(A[i:, i])) + i
        if i != max_row:
            A[[i, max_row]] = A[[max_row, i]]
            P[[i, max_row]] = P[[max_row, i]]
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]

        # Upper triangular matrix U
        for k in range(i, n):
            sum_upper = sum(L[i][j] * U[j][k] for j in range(i))
            U[i][k] = A[i][k] - sum_upper

        # Lower triangular matrix L
        for k in range(i, n):
            if i == k:
                L[i][i] = 1  # Diagonal as 1
            else:
                sum_lower = sum(L[k][j] * U[j][i] for j in range(i))
                L[k][i] = (A[k][i] - sum_lower) / U[i][i]

    return P, L, U

# Example usage:
A = np.array([[2, -1, -2],
              [-4, 6, 3],
              [-4, -2, 8]], dtype=float)

P, L, U = lu_decomposition_with_pivoting(A)
print("P:", P)
print("L:", L)
print("U:", U)


P: [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
L: [[ 1.    0.    0.  ]
 [ 1.    1.    0.  ]
 [-0.5  -0.25  1.  ]]
U: [[-4.    6.    3.  ]
 [ 0.   -8.    5.  ]
 [ 0.    0.    0.75]]


In [5]:
P.T @ L @ U

array([[ 2., -1., -2.],
       [-4.,  6.,  3.],
       [-4., -2.,  8.]])